In [9]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import re
import numpy as np
import yaml

In [10]:
config = yaml.load(open('config.yml'))
config

{'elasticsearch_host': 'localhost',
 'plotly_api_key': 'atqkijw50c',
 'plotly_user': 'intsco'}

In [12]:
tools.set_credentials_file(username=config['plotly_user'], api_key=config['plotly_api_key'])

In [13]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

client = Elasticsearch(hosts=config['elasticsearch_host'])

### SM dist results

In [579]:
s = (Search(using=client, index="sm") \
    #.filter("term", category="search") \
    #.query("match", ds_name="RB_a2s1"))
    .query("match", ds_name="Bruker//Rat//Brain//a2s1"))
    #.query(~Q("match", description="beta"))

In [580]:
#response = s.execute()
# r = list(response)[0]
response = s.scan()

In [581]:
dist_df = pd.DataFrame([(r.sf, r.adduct, r.msm, r.chaos, r.image_corr, r.pattern_match) for r in response],
                       columns=['sf', 'adduct', 'msm', 'moc', 'spat', 'spec']).set_index(['sf', 'adduct'])

In [582]:
dist_df[dist_df.index.duplicated()].shape

(0, 4)

In [583]:
dist_df.head()

,,msm,moc,spat,spec
sf,adduct,,,,
C40H80NO8P,+K,0.959347,0.999239,0.973018,0.986701
C27H44O,+H,0.956642,0.999265,0.975597,0.981292
C40H80NO8P,+Na,0.941901,0.999108,0.977463,0.964478
C44H86NO8P,+Na,0.918834,0.999335,0.935765,0.982560
C41H83N2O6P,+K,0.899275,0.999328,0.918035,0.980223


In [558]:
#dist_df = pd.read_csv('sm_dist_msm.csv').set_index(['sf', 'adduct']).drop(['formula_db', 'ds_name'], axis=1)

In [559]:
#dist_df.head()

### Ref pipeline results

In [584]:
pwd

u'/home/intsco/embl/tmp/rb_a2s1_debug'

In [585]:
ref_df = pd.read_csv('RB_a2s1_spatial_all_adducts_full_results.txt').dropna()
ref_df = ref_df[ref_df.adduct.isin(['+H', '+Na', '+K'])].set_index(['sf', 'adduct'])

In [586]:
ref_df.head()

mz       moc      spat      spec       msm
sf       adduct                                                    
C12H18O8 +K      329.063346  0.996271  0.037435  0.967960  0.036101
         +H      291.107447  0.990395  0.000000  0.965248  0.000000
         +Na     313.089418  0.996748  0.000000  0.964289  0.000000
C12H18O9 +K      345.058243  0.998403  0.038135  0.957136  0.036442
         +H      307.102382  0.995199  0.036683  0.959727  0.035037

In [587]:
ref_df[ref_df.index.duplicated()]

,,mz,moc,spat,spec,msm
sf,adduct,,,,,


In [590]:
ref_df.tail()

mz       moc      spat      spec       msm
sf         adduct                                                    
C9H17N4O9P +K      395.036494  0.991296  0.000000  0.973450  0.000000
           +Na     379.062553  0.994760  0.027157  0.974596  0.026328
C17H15NO   +K      288.078538  0.986694  0.009046  0.944659  0.008432
           +H      250.122653  0.991016  0.000000  0.942523  0.000000
           +Na     272.104605  0.993179  0.000000  0.952641  0.000000

### Plotting

In [595]:
plot_df = dist_df.join(ref_df, rsuffix='_ref', how='inner').dropna()
plot_df.shape

(15408, 9)

In [601]:
plot_df.head()

msm       moc      spat      spec           mz   moc_ref  spat_ref  spec_ref   msm_ref
sf              adduct                                                                                             
C102H170N2O22P2 +H      0.071172  0.989494  0.094174  0.763770  1839.182540  0.984320  0.166277  0.669270  0.109539
                +Na     0.169180  0.980158  0.216761  0.796288  1861.164463  0.989823  0.075237  0.674998  0.050268
C10Cl10O        +H      0.000000  0.000000  0.000000  0.000000   490.684687  0.997411  0.000000  0.599449  0.000000
C10Cl12         +H      0.000000  0.000000  0.000000  0.000000   546.624660  0.996009  0.000000  0.579306  0.000000
C10H10N2O       +K      0.000000  0.000000  0.000000  0.000000   213.042476  0.988157  0.003408  0.973550  0.003279

In [602]:
text_tmpl = '{}{}<br>Ref pipe: moc={:.3f} spat={:.3f} spec={:.3f}<br>SM dist: moc={:.3f} spat={:.3f} spec={:.3f}'

plot_df_H = plot_df.xs('+H', level='adduct')
text_H = plot_df_H.reset_index().apply(lambda r: text_tmpl.format(
        r.sf, '+H', r.moc, r.spat, r.spec, r.moc_ref, r.spat_ref, r.spec_ref), axis=1)

plot_df_K = plot_df.xs('+K', level='adduct')
text_K = plot_df_K.reset_index().apply(lambda r: text_tmpl.format(
        r.sf, '+K', r.moc, r.spat, r.spec, r.moc_ref, r.spat_ref, r.spec_ref), axis=1)

traceH = go.Scatter(
    x = plot_df_H['msm'],
    y = plot_df_H['msm_ref'],
    text = text_H,
    mode = 'markers',
    name = '+H'
)
traceK = go.Scatter(
    x = plot_df_K['msm'],
    y = plot_df_K['msm_ref'],
    text = text_K,
    mode = 'markers',
    name = '+K'
)

data = go.Data([traceH, traceK])
fig = Figure(data=data, layout = go.Layout(
    autosize=False,
    height=800,
    hovermode='closest',
    title='MSM values',
    width=800,
    xaxis=XAxis(
        autorange=True,
        range=[-0.05675070028979684, 1.0323925590539844],
        title='Reference MSM values',
        type='linear'
    ),
    yaxis=YAxis(
        autorange=True,
        range=[-0.0015978995361995152, 1.0312345837176764],
        title='SM engine MSM values',
        type='linear'
    )
))
py.iplot(fig, filename='ref_dist_msm_scatter')

In [603]:
tmp_df = plot_df.dropna()
np.corrcoef(tmp_df['msm'].values, tmp_df['msm_ref'].values)

array([[ 1.        ,  0.87062926],
       [ 0.87062926,  1.        ]])